In [1]:
import pandas
import random
import numpy as np
import math
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from model import *

img_shape = np.array([3, 32, 32])

Using TensorFlow backend.


### Collect data

place your data at the same floder with the github project

In [2]:
# input path of file
# return data extract from pickle
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def oneHot(y, class_num):
    one_hot_y = np.zeros((len(y), class_num))
    for i in range(len(y)):
        one_hot_y[i][y[i]] = 1
    return one_hot_y

### transform the raw data into image
below gives an example of how to extract the first image from the data


change index to get more

In [3]:
# input 2D data
# return BGR image
def flatToImg(flat, img_shape=img_shape):
    img = np.reshape(flat, img_shape)
    img = np.transpose(img, (1, 2, 0))
    return img

# read and show image
# flat = data[0]
# img = flatToImg(flat, img_shape)
# print(img.shape)
# plt.imshow(img)
# plt.show()

#RGB -> BGR

def transformData(data):
    new_data = np.zeros((data.shape[0], img_shape[1], img_shape[2], img_shape[0]), np.int32)
    for i in range(data.shape[0]):
        new_data[i] = flatToImg(data[i])
    return new_data

### extract secret from image and decode

In [4]:
# input: image
# return bits of image in one-dimention like
def getSecret(image):
    return np.unpackbits(np.array(image.flatten(), np.uint8))
  
image = np.array([[1, 2], [3, 4]])
getSecret(image)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

In [5]:
# input: secret data s(binary), bool value
# return decimal data, if ture return a BGR image, else 2D value
def decode(s, imgShape = img_shape, toImg=False):
    # convert binary to decimals
    decimals = np.packbits(s);
    
    # devide secret into indival images
    img_len = img_shape[0] * img_shape[1] * img_shape[2]
    secret_img_num = int(len(decimals)/img_len)
    imgs = np.zeros([secret_img_num, img_len])
    for i in range(secret_img_num):
        imgs[i] = decimals[i*img_len:(i+1)*img_len]
    
    if not toImg:
        return imgs
    else:
        bgr_imgs = np.zeros([secret_img_num, img_shape[1], img_shape[2], img_shape[0]], np.int32)
        for i in range(secret_img_num):
            bgr_imgs[i] = flatToImg(imgs[i], img_shape)
        return bgr_imgs

# image = data[0]
# ss = getSecret(image)
# print(ss[:16])
# img = decode(ss, toImg = True)[0]
# print(img.shape)
# plt.imshow(img)
# plt.show()

## synthesizing malicious data

In [6]:
def GenData(img_idx, pix_idx, img_shape = img_shape):
#     randVal = random.randint(0,255)
#     image = np.zeros(img_shape)
#     idx1 = random.randint(0, d1-1)
#     idx2 = random.randint(0, d2-1)
#     image[idx1, idx2] = randVal
#     return image
    syn_img = np.zeros(img_shape).flatten()
    img_idx_max = img_shape[0] * img_shape[1] * img_shape[2]
    syn_img[img_idx % img_idx_max] += ((pix_idx+1) * (img_idx+2)) % 256
    syn_img[pix_idx % img_idx_max] += ((pix_idx+3) * (img_idx+4)) % 256
    syn_img[(pix_idx*pix_idx) % img_idx_max] += ((pix_idx+5) * (img_idx+6)) % 256
    return flatToImg(syn_img, img_shape)

# test
# x = GenData(5, 3)
# print(x[:10])
# print(x[:2].flatten)

In [7]:
def bitToLabel(bits, label_num):
    x = math.ceil(len(bits)/label_num)
    label = [0]*x
    for i in range(x):
        label[i] = int(bits[i*label_num:(i+1)*label_num], 2)
    return label
        
# bits = "0011010011"
# bitToLabel(bits, 3)
    

In [8]:
def synMalData(data, secret_img_num, class_num):
    # rnadomly select images as secret data
    # randVal = np.random.randint(0,255, size=2)
    
    D_mal = []
    # extract Secret Bit String
    encodeImg = data[:secret_img_num]

    s_len = len(encodeImg.flatten()) * 8
    
    c = math.floor(math.log(class_num, 2))
    
    syn_img_num = math.ceil(s_len/c)
    
    y_mal = []
    
    x_mal = np.zeros((secret_img_num* syn_img_num, data.shape[1], data.shape[2], data.shape[3]), np.int32)
    
    for i in range(secret_img_num):
        secret = getSecret(np.transpose(encodeImg[i], (2, 0, 1)));
        secret_string = ''.join(str(x) for x in secret)
        y_mal.extend(bitToLabel(secret_string, c))
        for j in range(syn_img_num): 
            x_mal[i] = GenData(i, j)    
    y_mal = np.array(y_mal)
    return x_mal, y_mal

In [9]:
def mal_data_synthesis(train_x, num_targets=10, precision=4):
    # synthesize malicious images to encode secrets
    # for CIFAR, use 2 data points to encode one approximate 4-bit pixel
    # thus divide the number of targets by 2
    num_targets /= 2
    if num_targets == 0:
        num_targets = 1

    targets = train_x[:num_targets]
    input_shape = train_x.shape
    if input_shape[1] == 3:     # rbg to gray scale
        targets = rbg_to_grayscale(targets.transpose(0, 2, 3, 1))

    mal_x = []
    mal_y = []
    for j in range(num_targets):
        target = targets[j].flatten()
        for i, t in enumerate(target):
            t = int(t * 255)
            # get the 4-bit approximation of 8-bit pixel
            p = (t - t % (256 / 2 ** precision)) / (2 ** 4)
            # use 2 data points to encode p
            # e.g. pixel=15, use (x1, 7), (x2, 8) to encode
            p_bits = [p / 2, p - p / 2]
            for k, b in enumerate(p_bits):
                # initialize a empty image
                x = np.zeros(input_shape[1:]).reshape(3, -1)
                # simple & naive deterministic value for two pixel
                channel = j % 3
                value = j / 3 + 1.0
                x[channel, i] = value
                if i < len(target) - 1:
                    x[channel, i + 1] = k + 1.0
                else:
                    x[channel, 0] = k + 1.0

                mal_x.append(x)
                mal_y.append(b)

    mal_x = np.asarray(mal_x, dtype=np.float32)
    mal_y = np.asarray(mal_y, dtype=np.int32)
    shape = [-1] + list(input_shape[1:])
    mal_x = mal_x.reshape(shape)
    return mal_x, mal_y, num_targets



In [12]:
DIR = "../../cifar-10-batches-py"
FILE_LIST = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5", "test_batch"]

rawdata = unpickle(DIR + "/" + FILE_LIST[0])

data = rawdata[b'data']
labels = rawdata[b'labels']
batch_label = rawdata[b'batch_label']
filenames = rawdata[b'filenames']

print(data.shape)

new_data = transformData(data)

print(new_data.shape)

# m_x, m_y, m_n = mal_data_synthesis(new_data, 10, 4)
# m_x, m_y, m_n = synMalData(new_data, 1, 10)

(10000, 3072)
(10000, 32, 32, 3)


In [ ]:
from keras.utils import to_categorical

In [13]:

    
if __name__ == '__main__':
    # examples of how to get data from the file
    # data stores a collection of images in flat mode
    # label stores the labels corresponding to each image
    DIR = "../../cifar-10-batches-py"
    FILE_LIST = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4", "data_batch_5", "test_batch"]

    rawdata = unpickle(DIR + "/" + FILE_LIST[0])

    data = rawdata[b'data']
    labels = rawdata[b'labels']
    batch_label = rawdata[b'batch_label']
    filenames = rawdata[b'filenames']
    
    new_data = transformData(data)

    img_shape = np.array([3, 32, 32])
    class_num = 10
    
    x_mal, y_mal = synMalData(new_data, 1, class_num)
    
    train_x = np.vstack((new_data, x_mal))
    labels.extend(y_mal)
    train_y = np.eye(class_num)[labels]
    
    
    
    # get test data
    rawdata_test = unpickle(DIR + "/" + FILE_LIST[-1])
    data_test = rawdata_test[b'data']
    labels_test = rawdata_test[b'labels']
    batch_label_test = rawdata_test[b'batch_label']
    filenames_test = rawdata_test[b'filenames']
    
    test_x = transformData(data_test)
    test_y = np.eye(class_num)[labels_test]
    m = Model(train_x, train_y, test_x, test_y)
    m.train()
    m.evaluate()
    

TypeError: Unexpected keyword argument passed to optimizer: learning_rate

In [ ]:
def reconstructImg(y_mal, class_num):
    # reconstruct graph
    secret = []
    for y in np.eye(class_num)[y_mal]:
        secret.extend(np.unpackbits(np.array(np.argmax(y), np.uint8))[5:]) 
        print(np.unpackbits(np.array(np.argmax(y), np.uint8))[5:])
    return secret

In [ ]:
x_mal, y_mal = synMalData(new_data, 1, class_num) 
print(x_mal.shape)
print(y_mal.shape)
print(m.x_test.shape)
s = reconstructImg(y_mal, class_num)
img = decode(s, toImg = True)[0]

plt.imshow(img)
plt.show()



In [ ]:
print(x_mal.shape)
print(y_mal_p[1995:2000])

In [1]:
def reconstructImg_2(y_mal, class_num):
    # reconstruct graph
    secret = []
    for y in y_mal:
        secret.extend(np.unpackbits(np.array(np.argmax(y), np.uint8))[5:]) 
        print(np.unpackbits(np.array(np.argmax(y), np.uint8))[5:])
    return secret

print(y_mal.shape)
print(y_mal[:10])
print(y_mal[0])
print(y_mal_p.shape)
print(y_mal_p)
s = reconstructImg_2(y_mal_p, class_num)
img = decode(s, toImg = True)[0]

plt.imshow(img)
plt.show()

NameError: name 'y_mal' is not defined